# Modelo LB 2D
Este cuaderno permite simular un fluido mediante la soluciÃ³n de un modelo Navier-Stokes

Referencia:
[Lattice Boltzmann 2D](https://drive.google.com/open?id=0B0KBxf-IgVRzZ0w1QzZQRklaekJJai1DUGVHbVRrRWVNMlAw)

Empecemos por agregar **una imagen desde drive**

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Carpeta de imÃ¡genes
path = '/content/drive/My Drive/UDEC/PROYECTOLB/img/'

Paquetes necesarios

In [0]:
import numpy as np
import matplotlib.pyplot as plt

## 1. FunciÃ³n de inicializaciÃ³n
El modelo **D2Q9** consiste en un conjunto de velocidades por cada direcciÃ³n en cada punto de la grilla. La probabilidad que una partÃ­cula se mueva en cada direcciÃ³n viene dada de acuerdo a los pesos $w_i$. Por otro lado la probabilidad que cierta cantidad de masa se mueva hacia cada direcciÃ³n estÃ¡ dada por la funciÃ³n de distribuciÃ³n $f_i$.

Como se observa en la figura siguiente hay mÃ¡s probabilidad que la masa de una partÃ­cula se quede en reposo (direcciÃ³n 0), seguido por la probabilidad que la masa se mueva hacia la abajo (direcciÃ³n 4).

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-lboi{border-color:inherit;text-align:center;vertical-align:middle}
.tg .tg-fymr{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-g7sd{font-weight:bold;border-color:inherit;text-align:center;vertical-align:middle}
.tg .tg-0pky{border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-fymr">Velocidad</th>
    <th class="tg-fymr">Vector</th>
    <th class="tg-fymr">Pesos $w$</th>
    <th class="tg-g7sd">RepresentaciÃ³n</th>
  </tr>
  <tr>
    <td class="tg-0pky">$\vec{v}_0$</td>
    <td class="tg-0pky">(0,0)</td>
    <td class="tg-0pky">4/9</td>
    <td class="tg-lboi" rowspan="9"><img src="https://www.cb-geo.com/images/cb-geo/research/lbm/d2q9.png" alt="D2Q9" width="500"/></td>

  </tr>
  <tr>
    <td class="tg-0pky">$\vec{v}_1$</td>
    <td class="tg-0pky">(1,0)</td>
    <td class="tg-0pky">1/9</td>
  </tr>
  <tr>
    <td class="tg-0pky">$\vec{v}_2$</td>
    <td class="tg-0pky">(0,1)</td>
    <td class="tg-0pky">1/9</td>
  </tr>
  <tr>
    <td class="tg-0pky">$\vec{v}_3$</td>
    <td class="tg-0pky">(-1,0)</td>
    <td class="tg-0pky">1/9</td>
  </tr>
  <tr>
    <td class="tg-0pky">$\vec{v}_4$</td>
    <td class="tg-0pky">(0,-1)</td>
    <td class="tg-0pky">1/9</td>
  </tr>
  <tr>
    <td class="tg-0pky">$\vec{v}_5$</td>
    <td class="tg-0pky">(1,1)</td>
    <td class="tg-0pky">1/36</td>
  </tr>
  <tr>
    <td class="tg-0pky">$\vec{v}_6$</td>
    <td class="tg-0pky">(-1,1)</td>
    <td class="tg-0pky">1/36</td>
  </tr>
  <tr>
    <td class="tg-0pky">$\vec{v}_7$</td>
    <td class="tg-0pky">(-1,-1)</td>
    <td class="tg-0pky">1/36</td>
  </tr>
    <td class="tg-0pky">$\vec{v}_8$</td>
    <td class="tg-0pky">(1,-1)</td>
    <td class="tg-0pky">1/36</td>
  </tr>
</table>

Entre otras, la siguiente propiedad se asegura con los pesos y velocidades seleccionadas:
$$\sum_i w_iv_i = 0$$

La siguiente funciÃ³n inicializa algunas variables. En cada punto de la grilla se tienen los mismos 9 pesos $w_i$ y 9 vectores de velocidad $v_i$. Sin embargo la funciÃ³n de distribuciÃ³n es diferente en cada punto de la grilla. AdemÃ¡s se realizan algunas pruebas de validaciÃ³n.

In [0]:
def inicialice(Ly, Lx):
  '''
  Inicializa las variables del modelo
  ### Input
  Lx: Longitud horizontal
  Ly: Longitud vertical
  ### Output: (w, f, v)
  w:  Pesos                       w.shape = (9,)
  f:  FunciÃ³n de distribuciÃ³n     f.shape = (Ly,Lx,9)
  v:  Vectores de velocidad D2Q9  v.shape = (9,2)
  '''
  
  # Cada uno de los 9 pesos
  w = np.array([4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36])
  
  # La funciÃ³n de distribuciÃ³n tiene 3 dimensiones,
  # las dos primeras corresponden a cada punto de la grilla y
  # la Ãºltima las 9 direcciones de cada punto
  f = np.zeros([Ly, Lx, 9])
  
  # En total hay 9 velocidades y cada una tiene dos dimensiones (y,x)
  # Ojo! La dimensiÃ³n y es la primer componente y la dimensiÃ³n x la segunda,
  # este cambio es por la forma en la cual Python lee una matriz
  v = np.array([[0, 0], [0, 1], [1, 0], [0, -1], [-1, 0], [1, 1], [1, -1], [-1,-1], [-1,1]])

  return w, f, v

# Pruebas: Se crean variables de testeo para 
# comprobar el correcto funcionamiento de la funcion
Lx_test = 3
Ly_test = 4
f_test = np.array([
    [[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9]],
    [[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9]],
    [[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9]],
    [[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9]]
])
w, f, v = inicialice(Ly_test, Lx_test)

# Prueba de dimensionalidad de la funciÃ³n de distribuciÃ³n
assert(f_test.shape == f.shape)

# Prueba de dimensionalidad de las velocidades
assert(v.shape == (9, 2))

# Prueba de dimensionalidad de los pesos
assert(w.shape == (9,))

## 2. FunciÃ³n densidad

La densidad $\rho$ de partÃ­culas en cada punto de la grilla viene dado por la suma de la funciÃ³n de distribuciÃ³n en dicho punto: 

$$\rho = \sum_i f_i$$

In [0]:
def densidad (f):
  '''
  Calcula la densidad en cada punto del espacio.
  ### Input
  f: FunciÃ³n de distribuciÃ³n
  ### Output
  rho: Densidad   rho.shape = (Ly, Lx)
  '''
  # Se suma en la 3era dimension (axis = 2)
  rho = np.sum(f, axis = 2)
  return rho

# Prueba de la funciÃ³n
rho_test = densidad(f_test)

# Prueba de dimensionalidad de la densidad
assert(rho_test.shape == (Ly_test, Lx_test))

## 3. Velocidad centro de masa

Cada partÃ­cula se mueve en cada direcciÃ³n, la velocidad del centro de masa $\vec{U}$ representa la velocidad de cada punto de la grilla. Este viene dado por:
$$\vec{U}\rho = \sum_i f_i \vec{v}_i = f \cdot \vec{v}$$

In [0]:
def velocidad_cm (f, v, rho):
  '''
  Calcula la velocidad centro de masa en cada punto del espacio.
  ### Input
  f: FunciÃ³n de distribuciÃ³n
  v: DistribuciÃ³n de velocidades
  ### Output:
  U.shape = (Ly, Lx, 2)
  '''
  
  # Se realiza producto punto entre f y v.
  # la suma se realiza entre la 3era dimensiÃ³n de f y
  # la 1era dimensiÃ³n de v. En estas dimensiones cada 
  # variable coincide con 9 elementos
  U = np.tensordot(f, v, axes = ([2], [0]))

  # Luego se divide entre la densidad. Como en cada punto de la grilla 
  # la densidad es escalar mientras la velocidad es un vector de dos dim,
  # se hace broadcasting de la densidad, de esta forma cada componente
  # del vector se divide en la misma densidad.
  U = np.divide(U, rho[...,None])
  return U

# Prueba de dimensionalidad de la velocidad del centro de masa
rho = densidad(f_test)
U_test = velocidad_cm(f_test, v, rho)
assert(U_test.shape == (Ly_test, Lx_test, 2))

## 4. Producto entre el velocidad del c. masa y la distribuciÃ³n de velocidad
Se calcula el producto punto entre $U$ y $v_i$. Este valor indica la proyecciÃ³n de la velocidad del centro de masa en cada direcciÃ³n.
$$\vec{U}\cdot\vec{v}_i = U_x v_{ix} + U_y v_{iy} $$

In [0]:
def Udotv (U, v):
  '''
  Calcula la proyecciÃ³n de la velocidad del centro de masa 
  en cada punto de la grilla para cada direcciÃ³n i
  ### Input
  U: velocidad_cm                 U.shape = (Ly, Lx, 2)
  v: DistribuciÃ³n de velocidades  v.shape = (9, 2)
  ### Output
  Udotv: ProyecciÃ³n velocidad cm  Udotv.shape = (Ly, Lx, 9)
  '''

  # Producto punto entre U y v. Ambas variables son vectores
  # con componentes y, x. Se multiplican estas componentes y
  # se suman. Para U estas componentes se encuentran en la 
  # 3era dimensiÃ³n, mientras para v en la 2da.
  return np.tensordot(U, v, axes = ([2], [1]))

# Prueba de dimensionalidad del producto entre velocidad 
# del centro de masa y la distribuciÃ³n de velocidad
Udotv_test = Udotv(U_test, v)
assert(Udotv_test.shape == (Ly_test, Lx_test, 9))

## 5. FunciÃ³n de equilibrio
Cuando dos esferas duras chocan entre sÃ­, la direcciÃ³n y velocidad de salida de cada una es muy sensible a las condiciones iniciales. Sin embargo estas direcciones y velocidades se tienden a distribuir de una manera muy particular alrededor de una velocidad promedio despuÃ©s de un tiempo sufiente. Esta distribuciÃ³n partÃ­cular se conoce como *distribuciÃ³n de equilibrio $f^{eq}$*.

$$f^{eq}_i = w_i*\rho*\left[1+3(\vec{U}\cdot\vec{v_i})+\dfrac{9}{2}(\vec{U}\cdot\vec{v_i})^2-\dfrac{3}{2}\vec{U}\cdot\vec{U}\right]$$


In [0]:
def f_eq (w, rho, U, v):
  '''
  Calcula la funciÃ³n de equilibrio.
  ### Input
  w: Pesos                       w.shape = (9,)
  rho: Densidad                  rho.shape = (Ly, Lx)
  U: velocidad_cm                U.shape = (Ly, Lx, 2)
  v: DistribuciÃ³n de velocidades v.shape = (9, 2)
  ### Output: 
  f_eq: FunciÃ³n de equilibrio    f_eq.shape = (Ly, Lx, 9)
  '''
  # Se obtienen las dimensiones de la grilla
  Ly, Lx = rho.shape

  # Se inicializa la funciÃ³n de equilibrio
  f_eq = np.zeros([Ly, Lx, 9])
  
  # Se calcula la proyecciÃ³n de la velocidad 
  # del centro de masa en cada direcciÃ³n
  udotv  = Udotv(U, v)                # Udotv.shape = (Ly, Lx, 9)

  # Se calcula la energÃ­a cinÃ©tica en 
  # cada punto de la grilla. Para esto 
  # se eleva cada componente al cuadrado (3era dimensiÃ³n)
  # y se suma (Ux^2 + Uy^2)
  U2     = np.sum(U**2, axis = 2)     # U2.shape = (Ly, Lx)
  
  for i in np.arange(9):
      f_eq[...,i] = w[i] * rho * (1 + 3*udotv[...,i] + 9/2*(udotv[...,i])**2 - 3/2*U2)
      
  return f_eq

# Prueba de dimensionalidad
rho_test = densidad(f_test)
feq_test = f_eq(w,rho_test, U_test, v)
assert(feq_test.shape == f_test.shape)

## 6. Condiciones iniciales
Antes de realizar sucesivas iteraciones de la funciÃ³n de distribuciÃ³n, es necesario inicializarla de acuerdo a las condiciones iniciales y de frontera. La variable *celdas* contiene informaciÃ³n de los lugares donde la grilla tiene un *obstÃ¡culo* o es *libre*.

CondiciÃ³n | celdas
----------|------------
ObstÃ¡culo | 0
Libre     | 1

TambiÃ©n se debe indicar la densidad inicial del fluido ($\rho_0$) y la velocidad cm inicial ($U_0$).

In [0]:
def condiciones_iniciales (w, rho0, U0, v, celdas):
  '''
  Consigue las condiciones inciales
  ### Input
  w: Pesos                       w.shape = (9,)
  rho0: Densidad inicial         escalar
  U0: velocidad_cm inicial       escalar
  v: DistribuciÃ³n de velocidades v.shape = (9,2)
  celdas: Obstaculo:0, libre:1   celdas.shape = (Ly,Lx)
  ### Output: 
  U: Velocidad cm inicial        U.shape = (Ly, Lx, 2)
  rho: Densidad incial           rho.shape = (Ly, Lx)
  f: FunciÃ³n distr. inicial      f.shape = (Ly, Lx, 9)

  f_eq:  f_eq.shape = (Ly, Lx, 9)
  '''

  # Dimensiones de la grilla
  Ly, Lx = celdas.shape
  
  # Convierta la velocidad inicial a una arreglo numpy
  U0 = np.array(U0)

  # Inicialice con ceros la velocidad inicial
  U = np.zeros([Ly, Lx, 2])
  
  # Cuando hay obstaculo (celdas=0), la velocidad es cero,
  # sino se permanece igual
  U = celdas[..., None]*U0[None,:]

  # La densidad en toda la grilla es rho0
  rho = rho0*np.ones([Ly,Lx])

  # FunciÃ³n de equilibrio con las condiciones iniciales
  f = f_eq(w, rho, U, v)

  return U, rho, f

# Pruebas


## 8. Lectura de imagen
A partir de una imagen png blanco/negro se obtiene la variable celdas (negro = 0, blanco = 1).
La imagen puede ser obtenida desde drive con el comando drive.mount(). Se realizan dos pruebas:
1. Para comprobar la lectura correcta de la imagen
2. Para comprobar las velocidades iniciales

In [0]:
from PIL import Image
def lectura_imagen(imagen):
  '''
  Carga una imagen png en escala blanco/negro.
  Obstaculo (negro):  Celda = 0
  Libre (blanco):     Celda = 1
  ### Input
  imagen png (blanco/negro)
  ### Output
  matriz
  '''
  # Lee imagen
  img = Image.open(imagen)

  # Convierte a matriz numpy
  matriz = np.array(img)
  
  return matriz


#### Prueba grafica 1

# Escriba la ruta de la imagen
imagen_test = 'test.png'

# Convierte la imagen a una matriz
matriz_test = lectura_imagen(path + imagen_test)

# Realiza pruebas en test.png
# assert(matriz_test.shape == (9,13) and # Prueba de dimensionalidad
#        matriz_test[0,0]  == 0      and # Prueba en esquina sup-izq
#        matriz_test[0,11] == 0      and # Prueba en esquina sup-der
#        matriz_test[8,10] == 0      and # Prueba en esquina inf-izq
#        matriz_test[8,3]  == 0          # Prueba en esquina inf-der
#       )
plt.imshow(matriz_test)
plt.show()

In [0]:
#### Prueba grafica 2
# Carpeta de imÃ¡genes
fluido = 'test.png'

# Lectura de imagen
celdas_test = lectura_imagen(path + fluido)

# Variables de la grilla
Ly_test, Lx_test = celdas_test.shape
X = np.arange(Lx_test)
Y = np.arange(Ly_test)

# Inicializa variables
w, f, v = inicialice(Ly_test, Lx_test)

# Inicializa condiciones inciales y de frontera
# densidad inicial = 1, velocidad inicial = (0,1)
U_inicial_test, rho_inicial_test, f_inicial_test = \
condiciones_iniciales(w, 1, [0,1], v, celdas_test)

# Componentes de las velocidades
vely_test = U_inicial_test[...,0]
velx_test = U_inicial_test[...,1]

# Densidad
rho_test = densidad(f_inicial_test)

# Campo vectorial de velocidades
plt.quiver(X, Y, velx_test, vely_test)

# Mapa de densidad
plt.imshow(rho_test)
plt.colorbar()
plt.show()

## 7. EvoluciÃ³n

In [0]:
def evolucion (f, w, v, U, rho, tau, celdas):
  '''
  Actualiza la funcion de distribuciÃ³n
  ### Input
  f: Funcion de distribucion     f.shape = (Ly, Lx, 9)
  w: Pesos                       w.shape = (9,)
  v: DistribuciÃ³n de velocidades v.shape = (9, 2)
  U: velocidad_cm                U.shape = (Ly, Lx, 2)
  rho: Densidad                  rho.shape = (Ly, Lx)
  tau: Parametro de relajacion   escalar
  celdas: Obstaculo:0, libre:1   celdas.shape = (Ly,Lx)
  ### Output
  f_new: fun. distr. actualizada f_new.shape = (Ly, Lx, 9)
  '''

  # Dimensiones de la grilla
  Ly, Lx = rho.shape
  
  # Inicializa f_new a ceros
  f_new = np.zeros(f.shape)
  
  # Calcula funcion de equilibrio
  feq = f_eq(w, rho, U, v)
  
  # Iteraciones
  for y in np.arange(Ly):
    for x in np.arange(Lx):
      for i in np.arange(9):
        pos = (y, x, i)               # Posicion actual
        y_new = (y+v[i,0]+Ly)%Ly      # Coordenada y nueva, frontera periodica
        x_new = (x+v[i,1]+Lx)%Lx      # Coordenada x nueva, frontera periodica
        pos_new = (y_new, x_new, i)   # Posicion nueva
        if celdas[y_new ,x_new] == 1: # Libre
          f_new[pos_new] = f[pos] - 1.0/tau*(f[pos]-feq[pos])
        else:                         # Obstaculo
          f_new[pos_new] = f[pos_new] 
  
  return f_new

# Prueba de dimensionalidad con tau = 0.5
f_new_test = evolucion (f_inicial_test, w, v, \
                    U_inicial_test, rho_inicial_test, 0.5, celdas_test)
# assert(f_test.shape == f_inicial_test.shape)

# Prueba grafica
rho_new_test = densidad(f_new_test)
U_new_test = velocidad_cm(f_new_test, v, rho_inicial_test)

plt.quiver(X, Y, U_new_test[...,1], U_new_test[...,0])

# Mapa de densidad
plt.imshow(rho_new_test)
plt.colorbar()
plt.show()

## 9. SimulaciÃ³n fluido

In [0]:
# ParÃ¡metro de relajaciÃ³n
tau  = 0.51

# Condiciones iniciales
rho0 = 1             # Densidad = 1
U0   = [0,0.05]      # Velocidad hacia la derecha 0.05
fluido = 'vena.png'  # Archivo con obstaculos

# Numero de iteraciones
iteraciones = 1000

# Grilla
celdas = lectura_imagen(path + fluido)
Ly, Lx = celdas.shape

# Inicializacion
w, f, v   = inicialice(Ly, Lx)
U, rho, f = condiciones_iniciales(w, rho0, U0, v, celdas)

# Almacenamiento de resultados, 
# se utiliza la primer dimension para cada iteracion
rho_cache = np.zeros([iteraciones, Ly, Lx])
U_cache = np.zeros([iteraciones, Ly, Lx, 2])

# Iteraciones
for i in np.arange(iteraciones):
  # Actualizacion de la funcion de distribucion
  f = evolucion(f, w, v, U, rho, tau, celdas)

  # Actualizacion de la densidad y la velocidad
  rho = densidad(f)
  U   = velocidad_cm(f, v, rho)
  
  # Almacenamiento
  rho_cache[i,...] = rho
  U_cache[i,...] = U

## 10. AnimaciÃ³n

In [0]:
import matplotlib.animation as animation
from IPython.display import HTML

# Funcion de actualizacion de la animacion
def update_quiver(i, Q, U):
  Q.set_UVC(U[i,...,1], U[i,...,0])
  #ax.imshow(rho_cache[i,...])
  return Q

# Preparacion de la figura
fig, ax = plt.subplots(1,1)

# Figura inicial
# Q = ax.quiver(np.arange(Lx), np.arange(Ly), U_cache[100,...,1], U_cache[100,...,0], color = 'black')
# ax.imshow(rho_cache[100,...])
# fig.colorbar()
# fig.show()

plt.quiver(np.arange(Lx), np.arange(Ly), U_cache[100,...,1], U_cache[100,...,0], color = 'white')
plt.imshow(rho_cache[100,...])
plt.colorbar()
plt.show()

# Limites de dibujo
# ax.set_xlim(30, 70)
# ax.set_ylim(10, 50)

# Animacion
# anim = animation.FuncAnimation(fig, update_quiver, fargs=(Q, U_cache),
                              #  interval=20, blit=False, save_count=iteraciones)

# Conversion a HTML para cuaderno jupyter
# HTML(anim.to_html5_video())

## Diferencia de velocidad
$$\frac{\partial u_x}{\partial y} = \frac{u_x(x,y+1)-u_x(x,y-1)}{2}$$

In [0]:
def D_velocidad_cm (U):
  '''
  Calcula la velocidad centro de masa en cada punto del espacio.
  ### Input
  U: Velocidad centro de masa U.shape = (Ly, Lx, 2)
  ### Output:
  DU.shape = (Ly, Lx, 2)
  '''
  x = 1
  y = 2
  
  Ly, Lx, i = U.shape
  DUx = np.zeros([Ly,Lx,i])
  
  for y in range(Ly):
    for x in range(Lx):
      y_der = (y+1)%Ly
      y_izq = (y-1)%Lx
      DUx[y,x,1] = (U[y_der, x, 1] - U[y_izq,x,1])/2

  return DUx

# Prueba
DUx = D_velocidad_cm(U_test)
print(U_test)
print (DUx)
# Prueba de dimensionalidad de la velocidad del centro de masa


In [0]:
-1%4